In [2]:
# OS/ User specific 

TRAINING_DATASET_PATH = "/home/jcharlet/workspace/datascience/machine_learning_notes/data" + '/train.csv'
# TRAINING_DATASET_PATH = os.path.abspath(os.path.join( os.getcwd() , 'data')) + r'\train.csv'

### Kaggle project - Table of Contents
* 1 - Define the problem
* [Load data and displaying info](#load-data)
* 2 - Prepare Data
    * Identify features
        * Separate numerical from categorical features
        * Separate nominal and ordinal (from categorical features)
    * Clean data
        * Remove numerical features with missing values
        * Remove categorical features with missing values
        * drop outliers in numerical values # WIP
    * transform # TODO
        * transform categorical values #TODO
* 2 - Feature selection #WIP
    * Select features using random forest classifier #WIP
    
* 3 - Spot Check Algorithms
    * split dataset
    * train on multiple algorithms  # WIP

# Define the problem

From project https://www.kaggle.com/c/house-prices-advanced-regression-techniques

- Step 1: What is the problem? 

house prices are hard to predict, we want to know how much my beautiful flat is worth on the market in the USA

- Step 2: Why does the problem need to be solved? 

MONEY MONEY
and especially we need to learn Machine Learning

- Step 3: How would I solve the problem? 

housing agents do statistical studies.

Data pre-processing is very important. We have a lot of features, dirty data (missing values), numerical/ordinal/nominal values.

We'll follow the process from https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/

- 1 - Define the Problem
- 2 - Prepare Data
- 3 - Spot Check Algorithms
- 4 - Improve Results
- 5 - Present Results

# Load data <a class="anchor" id="load-data"></a>

In [3]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv(TRAINING_DATASET_PATH)


In [5]:
# print(df.info())


# Prepare data <a class="anchor" id="clean-data"></a>
## Identify features
### Separate numerical from categorical features

In [6]:
# Separating data features according to data type: numeric or string

X_numeric_labels = set(df._get_numeric_data().columns.tolist())
X_categorical_labels = set(df.columns.tolist()).difference(X_numeric_labels)

In [7]:
print('These are numerical features {} \n'.format(X_numeric_labels))
print('These are categorical features {} \n'.format(X_categorical_labels))


These are numerical features {'WoodDeckSF', 'GarageCars', '3SsnPorch', 'Fireplaces', 'FullBath', 'SalePrice', 'Id', 'LowQualFinSF', 'HalfBath', '2ndFlrSF', 'GarageArea', 'YearBuilt', 'MSSubClass', 'GarageYrBlt', 'ScreenPorch', 'BsmtFinSF1', 'LotArea', 'OverallCond', '1stFlrSF', 'BedroomAbvGr', 'TotRmsAbvGrd', 'BsmtHalfBath', 'PoolArea', 'MoSold', 'YearRemodAdd', 'OverallQual', 'TotalBsmtSF', 'KitchenAbvGr', 'GrLivArea', 'MasVnrArea', 'BsmtUnfSF', 'OpenPorchSF', 'YrSold', 'BsmtFinSF2', 'EnclosedPorch', 'BsmtFullBath', 'LotFrontage', 'MiscVal'} 

These are categorical features {'CentralAir', 'KitchenQual', 'BsmtFinType2', 'HouseStyle', 'BsmtExposure', 'LandContour', 'Functional', 'LandSlope', 'Exterior1st', 'Alley', 'Condition2', 'Electrical', 'MasVnrType', 'Utilities', 'MSZoning', 'GarageType', 'BsmtCond', 'PoolQC', 'GarageCond', 'LotShape', 'SaleCondition', 'PavedDrive', 'Foundation', 'Street', 'BldgType', 'Heating', 'SaleType', 'ExterCond', 'FireplaceQu', 'Fence', 'MiscFeature', 'Neig

### Separate nominal and ordinal (from categorical features)

In [8]:
# Separating categorical features in two sets: nominal and ordinal
# The values in features asociated with categorical ordinal feautures are: [Ex, Gd, GLQ, GdPrv, Fin] ... etc

categorical_ordinal_list_values = ['Ex', 'Gd', 'GLQ', 'GdPrv', 'Fin'] # There are not iall of the values. Here must be
# included all the possible values for the categorical ordinal features

X_categorical_nominal_labels = (df[list(X_categorical_labels)].drop(['GarageCond', 'BsmtFinType1', 'BsmtQual','MiscFeature','Alley','GarageType','BsmtFinType2',
         'BsmtExposure', 'FireplaceQu', 'MasVnrType', 'GarageQual', 'GarageFinish', 'Fence', 'PoolQC'],
        1).isin(categorical_ordinal_list_values) == False).all().loc[lambda df: df.values == True].axes

# The previous instruction returns the index of the categorical_nominal_values. I think we could include the following features:
#LandSlope has values: Gtl -> Gentle slopen; Mod -> Moderate Slope; Sev -> Severe Slope
#Functional has values: Typ -> Typical Functionality; Min1 -> Minor Deductions 1; Min2 -> Minor Deductions 2; Mod -> Moderate Deductions; Maj1 -> Major Deductions 1; Maj2 -> Major Deductions 2; Sev -> Severely Damaged; Sal -> Salvage only
#PavedDrive has values:   Y -> Paved ; P -> Partial Pavement; N -> Dirt/Gravel



In [9]:
print(X_categorical_nominal_labels)
print()
print(X_categorical_nominal_labels[0].tolist())

[Index(['CentralAir', 'HouseStyle', 'LandContour', 'Functional', 'LandSlope',
       'Exterior1st', 'Condition2', 'Electrical', 'Utilities', 'MSZoning',
       'LotShape', 'SaleCondition', 'PavedDrive', 'Foundation', 'Street',
       'BldgType', 'Heating', 'SaleType', 'Neighborhood', 'LotConfig',
       'Condition1', 'Exterior2nd', 'RoofMatl', 'RoofStyle'],
      dtype='object')]

['CentralAir', 'HouseStyle', 'LandContour', 'Functional', 'LandSlope', 'Exterior1st', 'Condition2', 'Electrical', 'Utilities', 'MSZoning', 'LotShape', 'SaleCondition', 'PavedDrive', 'Foundation', 'Street', 'BldgType', 'Heating', 'SaleType', 'Neighborhood', 'LotConfig', 'Condition1', 'Exterior2nd', 'RoofMatl', 'RoofStyle']


### Remarks
<p> From the set of set of **X_categorical_labels** the set of **X_categorical_nominal_labels** is extracted to get 
the set of **X_categorical_ordinal_labels**</p>

* On the set of **X_categorical_ordinal_labels** we can apply pag 104 **mapping strategy**

* On the set of **X_categorical_nominal_labels** we can apply **one-hot encoding strategy** pag 106

In [10]:
X_categorical_ordinal_labels = set(X_categorical_labels).difference(set(X_categorical_nominal_labels[0].tolist()))

In [11]:
# print(X_categorical_labels)

In [12]:
# df[list(X_categorical_ordinal_labels)]

## Clean data
### Remove numerical features with missing values

#### Some considerations


* <p> Page 107 of "our book":  "After executing the preceding code, the first column of the NumPy array X now holds the new
colour values, which are encoded as follows:
blue to 0
green to 1
red to 2
If we stop at this point and feed the array to our classifier, *we will make one of the most common 
mistakes in dealing with categorical data*. Can you spot the problem? *
Although the colour values do not come in any particular order, a learning algorithm will now assume
that green is larger than blue, and red is larger than green. 
Although this assumption is incorrect, the algorithm could still produce useful results. 
However, those results would not be optimal* "</p>

* <p> In the numerical feature variables there is the possibility that there are categorical numerical features variables. How to identify this kinf of features ? 
 I think the main problem is what strategy to use to replace values ? median, mode or mean ? </p>
 

In [13]:
df_cleaned=df.copy()


#Let's check how many numerical features have missing values

# print(df[df._get_numeric_data().columns.tolist()].isnull().sum())

# Dropping columns: ['Id', 'LotFrontage', 'GarageYrBlt', 'SalePrice',] from numerical features

# dropping Id and 
df_cleaned= df_cleaned._get_numeric_data().drop(['Id'],1)
X_numerical_not_missing_values_labels=df_cleaned[df_cleaned._get_numeric_data().columns.tolist()].isnull().sum().loc[lambda df: df.values == 0].axes[0].tolist()
df_cleaned=df[X_numerical_not_missing_values_labels]
# df_cleaned = df_cleaned._get_numeric_data().drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'],1)

# print(df_cleaned[df_cleaned._get_numeric_data().columns.tolist()].isnull().sum())

### Remove categorical features with missing values

In [14]:
# Checking for missing values in categorical values in the data set

# print(df[list(X_categorical_labels)].isnull().sum())

# Dropping categorical features with missing values
# The following instruction returns the list of features with no missing values

X_categorical_not_missing_values_labels = df[list(X_categorical_labels)].isnull().sum().loc[lambda df: df.values == 0].axes[0].tolist()
df_cleaned=df[X_categorical_not_missing_values_labels]
# df[list(X_categorical_labels)].drop(X_categorical_not_missing_values_labels, 1)


### Drop outliers in numerical values # WIP

In [15]:
def drop_outliers(df, field_name):
    distance = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
    df.drop(df[df[field_name] > distance + np.percentile(df[field_name], 75)].index, inplace=True)
    df.drop(df[df[field_name] < np.percentile(df[field_name], 25) - distance].index, inplace=True)

# drop_outliers(df_cleaned, 'LotArea')    
df_cleaned_no_outlier = df.copy()
for feature in X_numerical_not_missing_values_labels:
    drop_outliers(df_cleaned_no_outlier, feature)    

In [16]:
print("nb of rows with outliers:{}, without: {}".format(df.shape[0],df_cleaned_no_outlier.shape[0]))


nb of rows with outliers:1460, without: 584


We are currently removing far too many items, to review later.
can be done with sklearn anyway http://scikit-learn.org/stable/modules/outlier_detection.html

# Feature Selection

In [17]:
# feature_columns=filter(lambda x: x !="SalePrice" and x!="Id", df.columns)
# feature_numerical_columns = df[feature_columns].select_dtypes(include=[np.number]).columns.tolist()
feature_numerical_columns=X_numerical_not_missing_values_labels
prediction_column="SalePrice"
# # print(feature_columns)
X = df[feature_numerical_columns]
Y=df[[prediction_column]];

In [18]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier();
model = model.fit(X, Y)    
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("label {} : {}".format(feature_numerical_columns[f],importances[indices[f]]))
# print(indices)

# plt.bar(range(X.shape[1]),importances[indices],align='center')
# plt.xticks(range(X.shape[1]),feature_numerical_columns,rotation=90);
# plt.rcParams["figure.figsize"] = (17,5)


label MSSubClass : 0.11583775394852835
label LotArea : 0.059596391198568036
label OverallQual : 0.05929042687921961
label OverallCond : 0.05535591335180703
label YearBuilt : 0.05339928648765881
label YearRemodAdd : 0.05265642588156977
label BsmtFinSF1 : 0.051037160212856605
label BsmtFinSF2 : 0.049344319785879595
label BsmtUnfSF : 0.04507117617143287
label TotalBsmtSF : 0.04410527899239075
label 1stFlrSF : 0.04333008998472061
label 2ndFlrSF : 0.04082381491749085
label LowQualFinSF : 0.035963727230624756
label GrLivArea : 0.03170844144728096
label BsmtFullBath : 0.02767985328649026
label BsmtHalfBath : 0.02637204359092779
label FullBath : 0.02510182064705983
label HalfBath : 0.02093876563531272
label BedroomAbvGr : 0.020709375681647144
label KitchenAbvGr : 0.02040290304830243
label TotRmsAbvGrd : 0.016554703079649347
label Fireplaces : 0.01585286962539515
label GarageCars : 0.014978839352761194
label GarageArea : 0.012217462272628137
label WoodDeckSF : 0.01217825478545741
label OpenPorc

To draw graph and select features

# Spot Check Algorithms
## Split dataset

In [19]:
from sklearn.model_selection import train_test_split

X = df[X_numerical_not_missing_values_labels]
Y=df[["SalePrice"]];
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.33,random_state=42)
print(Y_train.head())
X_train.head()

      SalePrice
615      137500
613      147000
1303     232000
486      156000
561      170000


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
615,85,8800,6,7,1963,1963,763,0,173,936,...,120,0,0,0,0,0,0,5,2010,137500
613,20,8402,5,5,2007,2007,206,0,914,1120,...,0,30,0,0,0,0,0,12,2007,147000
1303,20,8688,7,5,2005,2005,0,0,1616,1616,...,208,59,0,0,0,0,0,4,2006,232000
486,20,10289,5,7,1965,1965,836,0,237,1073,...,0,0,0,0,0,0,0,6,2007,156000
561,20,10010,5,5,1974,1975,1071,123,195,1389,...,240,38,0,0,0,0,0,4,2006,170000


## Train on multiple models

In [20]:

#training
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 

from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing

models = []
models.append(("SVC",SVC()))
models.append(("LinearSVC",LinearSVC()))
models.append(("KNeighbors",KNeighborsClassifier()))
models.append(("DecisionTree",DecisionTreeClassifier()))
models.append(("RandomForest",RandomForestClassifier()))

modelsWhichNeedNormalization = []
modelsWhichNeedNormalization.append(("LogisticRegression",LogisticRegression()))
modelsWhichNeedNormalization.append(("MLPClassifier",MLPClassifier(solver='lbfgs', random_state=0)))
modelsWhichNeedNormalization.append(("LinearRegression", LinearRegression()))
# models

In [39]:
print("5 expected values: {}\n".format(Y_test[0:5].values))
for name,model in models:
#     Train the model using the training sets
    model.fit(X_train, Y_train)

    # Make predictions using the testing set
    y_pre = model.predict(X_test)
    print("Training on {}\n   mean squared log error: {}\n".format(name,mean_squared_log_error(Y_test, y_pre)))
    print("   5 predicted values: {}".format(y_pre[0:5]))

5 expected values: [[154500]
 [325000]
 [115000]
 [159000]
 [315500]]

Training on SVC
   mean squared log error: 0.1979351767720077

   5 predicted values: [140000 140000 140000 140000 140000]
Training on LinearSVC
   mean squared log error: 1.195296054472813

   5 predicted values: [60000 60000 60000 60000 60000]
Training on KNeighbors
   mean squared log error: 0.0013517972574133067

   5 predicted values: [155000 324000 115000 157000 318000]
Training on DecisionTree
   mean squared log error: 0.006966509708546483

   5 predicted values: [154000 335000 115000 159500 319900]
Training on RandomForest
   mean squared log error: 0.07289343355858952

   5 predicted values: [149000 412500 113000 130000 176000]


In [22]:

# for name,model in modelsWhichNeedNormalization:
# #     Train the model using the training sets
#     X_train_scaled = preprocessing.scale(X_train)
#     model.fit(X_train_scaled, Y_train)

#     # Make predictions using the testing set
#     X_test_scaled = preprocessing.scale(X_test)
#     y_pre = model.predict(X_test_scaled)
#     print("Training on {}: {}".format(name,mean_squared_log_error(Y_test, y_pre)))

# crash on Jeremie's laptop